In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from traceback import print_exception

%matplotlib inline

In [75]:
# Data loading. Refer to the mnl_exploration.ipynb file for details on how user-level features are generated.

df = pd.read_csv('../data/final_modeling_data_02142024.csv')

/var/folders/4x/l9lw50rn7qvf79m01f21x70mlpd6gh/T/ipykernel_84693/3166671539.py:3: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/final_modeling_data_02142024.csv')


In [76]:
print("Shape: ", df.shape)

Shape:  (80690, 105)


In [77]:
df.drop_duplicates(inplace=True)

### Add weather info

In [78]:
df.start_fmt_time = pd.to_datetime(df.start_fmt_time, utc=True).dt.tz_convert('America/Denver')

In [79]:
# We now need weather data. To get that, we download data from OpenMeteo for the given dates.
df.start_fmt_time.min(), df.start_fmt_time.max()

(Timestamp('2020-11-02 18:48:13.450000-0700', tz='America/Denver'),
 Timestamp('2022-12-30 23:33:27.147785-0700', tz='America/Denver'))

In [80]:
# Downloaded using OpenMeteo API.
weather_df = pd.read_csv('../data/denver_weather_data.csv')

In [81]:
weather_df.time = pd.to_datetime(weather_df.time, utc=True).dt.tz_convert('America/Denver')

In [82]:
original_shape = df.shape[0]

# Round start time to nearest hour.
df['start_time_rounded'] = df.start_fmt_time.dt.round(
    'H', ambiguous='infer', nonexistent='shift_backward'
)

In [83]:
# True intersection
df = df.merge(right=weather_df, left_on='start_time_rounded', right_on='time', how='left')

In [84]:
# Drop rows where weather data is NA.
df.dropna(subset=['temperature_2m (°F)','relative_humidity_2m (%)', 'dew_point_2m (°F)', 'rain (inch)',
                  'snowfall (inch)', 'wind_speed_10m (mp/h)', 'wind_gusts_10m (mp/h)'], inplace=True)

In [85]:
print(f"Old shape: {original_shape}. new shape: {df.shape[0]}")

Old shape: 80690. new shape: 80688


In [86]:
df.drop(columns=['start_time_rounded', 'time'], inplace=True)

### Creating the dummy variables for the available modes

In [87]:
df.isna().sum().to_dict()

{'source': 0,
 'end_ts': 0,
 'end_fmt_time': 0,
 'end_loc': 0,
 'raw_trip': 0,
 'start_ts': 0,
 'start_fmt_time': 0,
 'start_loc': 0,
 'duration': 0,
 'distance': 0,
 'start_place': 0,
 'end_place': 0,
 'cleaned_trip': 0,
 'inferred_labels': 216,
 'inferred_trip': 216,
 'expectation': 216,
 'confidence_threshold': 216,
 'expected_trip': 216,
 'user_input': 0,
 'section_modes': 0,
 'section_distances': 0,
 'start:year': 0,
 'start:month': 0,
 'start:day': 0,
 'start:hour': 0,
 'start_local_dt_minute': 0,
 'start_local_dt_second': 0,
 'start_local_dt_weekday': 0,
 'start_local_dt_timezone': 0,
 'end:year': 0,
 'end:month': 0,
 'end:day': 0,
 'end:hour': 0,
 'end_local_dt_minute': 0,
 'end_local_dt_second': 0,
 'end_local_dt_weekday': 0,
 'end_local_dt_timezone': 0,
 '_id': 0,
 'user_id': 0,
 'metadata_write_ts': 0,
 'additions': 80686,
 'mode_confirm': 337,
 'purpose_confirm': 527,
 'distance_miles': 0,
 'Mode_confirm': 0,
 'Replaced_mode': 0,
 'Trip_purpose': 0,
 'original_user_id': 0,


In [88]:
# | [d1, d2, d3] | [t1, t2, t3] | [m1, m2, m3] |


def compute_argmax(df: pd.DataFrame):
    # Create the two columns.
    df[['section_distance_argmax', 'section_duration_argmax', 'section_mode_argmax', 'section_coordinates_argmax']] = None
    rows = list()

    for ix, row in df.iterrows():
        row_dict = row.to_dict()
        
        try:
            parsed_distances = ast.literal_eval(row_dict['section_distances'])
            parsed_durations = ast.literal_eval(row_dict['section_durations'])
            parsed_modes = ast.literal_eval(row_dict['section_modes'])

            argmax_ix = np.argmax(parsed_distances)
            
            # Pick the argmax and scale to miles. (meters -> miles)
            row_dict['section_distance_argmax'] = parsed_distances[argmax_ix] * 0.0006213712
            
            # Pick the argmax and scale to minutes.
            row_dict['section_duration_argmax'] = parsed_durations[argmax_ix] / 60
            
            # Pick the argmax mode.
            row_dict['section_mode_argmax'] = parsed_modes[argmax_ix]
            
            row_dict['mark'] = False

        except Exception as e:
            row_dict['mark'] = True

        rows.append(row_dict)

    return pd.DataFrame(rows)

In [89]:
# Let's plot the mode-wise durations as a function of distance.
df = compute_argmax(df)

In [90]:
df.drop(columns=['section_distances', 'section_durations', 'section_modes'], inplace=True)

In [91]:
df = df.loc[~df.mark, :].reset_index(drop=True, inplace=False).drop(columns=['mark'], inplace=False)

In [92]:
# Drop instances where duration/distance is unusable.
df.drop(
    index=df.loc[(df.section_distance_argmax <= 0) | (df.section_duration_argmax <= 0), :].index,
    inplace=False
).reset_index(drop=True, inplace=True)

In [93]:
# bus, train, bicycling, walking, car
# split-apply-combine
def drop_outliers(df: pd.DataFrame) -> pd.DataFrame:
    def filter_by_percentiles(group):
        distance_low = group['section_distance_argmax'].quantile(0.1)
        distance_high = group['section_distance_argmax'].quantile(0.9)
        duration_low = group['section_duration_argmax'].quantile(0.1)
        duration_high = group['section_duration_argmax'].quantile(0.9)
        
        l1_filter = group[
            (group['section_distance_argmax'] >= distance_low) &
            (group['section_distance_argmax'] <= distance_high)
        ].reset_index(drop=True)
        
        l2_filter = l1_filter[
            (l1_filter['section_duration_argmax'] >= duration_low) &
            (l1_filter['section_duration_argmax'] <= duration_high)
        ].reset_index(drop=True)
        
        return l2_filter
    
    return df.groupby('section_mode_argmax').apply(filter_by_percentiles).reset_index(drop=True)

In [94]:
filtered_df = drop_outliers(df)

# Ideal speed. distance/time.
filtered_df['mph'] = (
    (filtered_df['section_distance_argmax'] * 60.)/filtered_df['section_duration_argmax']
)

In [95]:
def filter_mph(df: pd.DataFrame) -> pd.DataFrame:
    
    MPH_THRESHOLDS = {
        # https://www.sciencedirect.com/science/article/pii/S2210670718304682
        'bicycling': 15.,
        # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7806575/
        'walking': 2.93
    }
    
    def custom_filter(group):
        # Drop data specified in the dict manually.
        if group.name in MPH_THRESHOLDS.keys():
            f_df = group[group['mph'] <= MPH_THRESHOLDS[group.name]]
        else:
            mph_low = group['mph'].quantile(0.1)
            mph_high = group['mph'].quantile(0.9)

            f_df = group[(group['mph'] >= mph_low) & (group['mph'] <= mph_high)]
        
        return f_df
    
    return df.groupby('section_mode_argmax').apply(custom_filter).reset_index(drop=True)

In [96]:
filtered_df = filter_mph(filtered_df)

In [97]:
filtered_df.groupby('section_mode_argmax')[['section_distance_argmax', 'section_duration_argmax']].describe()

section_distance_argmax                                   \
                                      count       mean        std        min   
section_mode_argmax                                                            
bicycling                            7196.0   2.147281   0.888086   0.802221   
bus                                   290.0   1.432493   0.618867   0.372875   
car                                 24744.0   4.132410   2.601994   0.911446   
no_sensed                            1681.0   2.677326   2.365448   0.178071   
subway                                  7.0  23.441479  10.540687  13.113128   
train                                  63.0  12.262300   4.021215   4.322110   
walking                             12072.0   0.475999   0.368141   0.120082   

                                                                 \
                           25%        50%        75%        max   
section_mode_argmax                                               
bicycling             1.500171   1.955479   2.585763   4.892137   
bus                   0.940211   1.486744   1.917582   3.016720   
car                   2.192661   3.429022   5.350170  14.688077   
no_sensed             1.026175   1.954673   3.547600  13.554013   
subway               14.804241  23.587655  27.444590  42.891905   
train                10.030392  10.691830  15.446952  22.820908   
walking               0.221963   0.348252   0.594541   2.208408   

                    section_duration_argmax                                   \
                                      count       mean        std        min   
section_mode_argmax                                                            
bicycling                            7196.0  13.754313   5.148564   5.886639   
bus                                   290.0   8.960115   3.615953   2.963177   
car                                 24744.0  13.972971   6.666358   5.672830   
no_sensed                            1681.0  15.967264  12.112498   1.079868   
subway                                  7.0  41.873968  17.610572  23.757772   
train                                  63.0  29.722451  10.548372  12.519296   
walking                             12072.0  19.668879  13.049073   5.284233   

                                                                 
                           25%        50%        75%        max  
section_mode_argmax                                              
bicycling             9.927844  12.655730  16.655780  29.141947  
bus                   6.496972   8.240974  10.668935  20.585433  
car                   8.779016  12.123820  17.505811  35.306784  
no_sensed             7.166667  12.307400  21.626077  55.445744  
subway               29.866407  34.400000  51.801433  71.624326  
train                21.976672  26.400000  38.511937  55.713396  
walking               9.391205  15.457291  26.251433  60.423767

In [98]:
filtered_df.groupby('section_mode_argmax')[['mph']].describe()

mph                                              \
                       count       mean        std        min        25%   
section_mode_argmax                                                        
bicycling             7196.0   9.550366   2.371204   2.817580   7.840111   
bus                    290.0   9.848150   3.112711   4.264324   7.275453   
car                  24744.0  17.354381   5.587948   8.566853  12.716558   
no_sensed             1681.0  10.299587   4.514342   4.608316   6.571906   
subway                   7.0  34.708676  12.047924  24.457144  26.517639   
train                   63.0  25.095616   3.084808  19.883468  22.699687   
walking              12072.0   1.621529   0.745358   0.131592   0.998944   

                                                      
                           50%        75%        max  
section_mode_argmax                                   
bicycling             9.587994  11.239639  14.998472  
bus                   9.853532  12.421118  15.596840  
car                  16.579894  21.470495  30.036223  
no_sensed             9.213382  13.050427  22.598194  
subway               32.866807  36.515566  59.570373  
train                24.746441  27.292824  31.597822  
walking               1.616822   2.261444   2.929953

In [99]:
print(f"Dropped {df.shape[0] - filtered_df.shape[0]} rows.")

Dropped 34542 rows.


In [100]:
filtered_df.Replaced_mode.unique()

array(['Regular Bike', 'Other', 'No Travel', 'Gas Car, drove alone',
       'Gas Car, with others', 'Walk', 'Taxi/Uber/Lyft', 'Train',
       'Bikeshare', 'Bus', 'Skate board', 'Free Shuttle', 'Scooter share',
       'E-bike', 'Not a Trip'], dtype=object)

In [101]:
## Define the mapping strategy

"""
p_micro: Personal micromobility
s_micro: Shared micromobility
s_car: Shared car
car: Car/rental car
transit: train + bus
no_trip: No vehicle / None
ridehail: Uber/Lyft
"""

available = {
    'Bicycle': 'p_micro',
    'Do not have vehicle': 'unknown',
    'Get a ride from a friend or family member': 's_car',
    'None': 'no_trip',
    'Public transportation (bus, subway, light rail, etc.)': 'transit',
    'Rental car (including Zipcar/ Car2Go)': 'car',
    'Shared bicycle or scooter': 's_micro',
    'Skateboard': 'p_micro',
    'Taxi (regular taxi, Uber, Lyft, etc)': 'ridehail',
    'Walk/roll': 'walk',
    'Prefer not to say': 'unknown'
}

# Map each sensed mode to the binary indicators.
section_mode_mapping = {
    'bicycling': ['p_micro', 's_micro'],
    'bus': ['transit'],
    'car': ['s_car', 'car', 'ridehail'],
    'no_sensed': ['unknown'],
    'train': ['transit'],
    'walking': ['walk'],
    'subway': ['transit']
}

# For target - mode_confirm
# target_mapping = {
#     'Regular Bike': 'p_micro',
#     'Walk': 'walk',
#     'Gas Car, with others': 's_car',
#     'Gas Car, drove alone': 'car',
#     'Bikeshare': 's_micro',
#     'Other': 'unknown',
#     'Bus': 'transit',
#     'Not a Trip': 'no_trip',
#     'E-bike': 'p_micro',
#     'Train': 'transit',
#     'Taxi/Uber/Lyft': 'ridehail',
#     'Free Shuttle': 'transit',
#     'Scooter share': 's_micro',
#     'Skate board': 'p_micro'
# }

# For target - replaced_mode
target_mapping = {
    'Regular Bike': 'p_micro', 
    'Walk': 'walk', 
    'No Travel': 'no_trip', 
    'Other': 'unknown',
    'Gas Car, with others': 's_car',
    'Gas Car, drove alone': 'car',
    'Train': 'transit',
    'Scooter share': 's_micro', 
    'Taxi/Uber/Lyft': 'ridehail', 
    'Free Shuttle': 'transit', 
    'Skate board': 'p_micro',
    'Bikeshare': 's_micro',
    'Bus': 'transit',
    'Not a Trip': 'no_trip',
    'E-bike': 'p_micro'
}

# # These values were obtained using a simple linear regression model. Specifics can be found in time_distance_estimation.ipynb
# duration_dict = {
#     'walk': {
#         'intercept': 14.09698152519682,
#         'coef':  19.153802211061134
#     },
#     'p_micro': {
#         'intercept':  4.753590860550791,
#         'coef':  4.2723965916811935
#     },
#     's_micro': {
#         'intercept':  4.753590860550791,
#         'coef':  4.2723965916811935
#     },
#     's_car': {
#         'intercept':  10.515751350707365, 
#         'coef':  1.112111276629702
#     },
#     'car': {
#         'intercept':  10.515751350707365, 
#         'coef':  1.112111276629702
#     },
#     'ridehail': {
#         'intercept':  10.515751350707365, 
#         'coef':  1.112111276629702
#     },
#     'transit': {
#         'intercept':  7.5000351741492395,
#         'coef':  1.839562017986485
#     },
#     'unknown': {
#         'intercept':  9.423026094931313,
#         'coef':  2.640429444466706
#     },
#     'no_trip': {
#         'intercept': 0.,
#         'coef': 0.
#     }
# }

ordinal_mapping = {'p_micro': 1, 'no_trip': 2, 's_car': 3, 'transit': 4, 'car': 5, 's_micro': 6, 'ridehail': 7, 'walk': 8, 'unknown': 9}
reverse_ordinal_mapping = {v:k for k,v in ordinal_mapping.items()}

print(ordinal_mapping)

{'p_micro': 1, 'no_trip': 2, 's_car': 3, 'transit': 4, 'car': 5, 's_micro': 6, 'ridehail': 7, 'walk': 8, 'unknown': 9}


In [102]:
def generate_available_features(
    df: pd.DataFrame, av: dict, sm: dict, target_mapper: dict
):
    """
    This method generates the wide-form dummy features for available modes.
    The current implementation uses (section_mode_argmax OR available_modes) to indicate mode availability.
    """

    # Reset indices.
    df = df.reset_index(drop=True, inplace=False)

    columns = list(set(av.values())) + ['unknown']

    # Create the columns along with the 'chosen' column.
    df[columns] = 0

    row_dicts = list()

    for i, row in df.iterrows():
        row_dict = row.to_dict()

        # Access the available modes. Split on ; and strip all the elements.
        available_modes = [x.strip() for x in str(row_dict['available_modes']).split(';')]
        
        # Set all the available modes associated with the chosen modes too.
        # Toggle this flag off if you don't want the target to be considered as an available mode.
        # row_dict[row_dict['chosen']] = 1
        
        row_dict['target'] = target_mapper[row_dict['target']]
        
        # Update the available modes.
        for available_mode in available_modes:
            row_dict[av[available_mode]] = 1
        
        # Now, look at the section_mode_argmax as a proxy for the confirmed mode.
        chosen_modes = sm[row_dict['section_mode_argmax']]

        # Determine the length of the mapping.
        n = len(chosen_modes)

        # # Go ahead and modify the current dict with the first value.
        # row_dict['chosen'] = ordinal_mapper[chosen_modes[0]]
        # row_dict['primary_mode'] = 1

        # Also update the available modes.
        for chosen_mode in chosen_modes:
            row_dict[chosen_mode] = 1

        # # Append the first row to the list.
        row_dicts.append(row_dict)
        
    constructed = pd.DataFrame(row_dicts)
    constructed.rename(columns=dict([(c, 'av_'+c) for c in target_mapper.keys()]), inplace=True)

    return constructed

```language=python
{'p_micro': 1, 'no_trip': 2, 's_car': 3, 'transit': 4, 'car': 5, 's_micro': 6, 'ridehail': 7, 'walk': 8, 'unknown': 9}
```

In [103]:
def generate_target_variable(df: pd.DataFrame, mapper: dict):
    df['target'] = df['Replaced_mode'].apply(lambda x: mapper[x])
    df.drop(columns=['Replaced_mode'], inplace=True)
    return df

filtered_df = generate_target_variable(filtered_df, target_mapping)

In [104]:
# Generate wide-format availability indicators..
av_df = generate_available_features(filtered_df, available, section_mode_mapping, ordinal_mapping)

# Collapse train and bus into 'transit'.
av_df.loc[av_df.section_mode_argmax.isin(['bus', 'train', 'subway']), 'section_mode_argmax'] = 'transit'

In [105]:
av_df.section_mode_argmax.unique()

array(['bicycling', 'transit', 'car', 'no_sensed', 'walking'],
      dtype=object)

In [106]:
display(av_df.head())

,source,end_ts,end_fmt_time,end_loc,raw_trip,start_ts,start_fmt_time,start_loc,duration,distance,...,target,av_s_micro,av_walk,av_no_trip,av_unknown,av_p_micro,av_car,av_transit,av_s_car,av_ridehail
0,DwellSegmentationTimeFilter,1.619140e+09,2021-04-22 18:59:39-06:00,"{'type': 'Point', 'coordinates': [-105.1002447...",6082e47f27dcf1f393b70dd8,1.619139e+09,2021-04-22 18:48:48.365291-06:00,"{'type': 'Point', 'coordinates': [-105.0772567...",10.843912,1995.541857,...,1,1,0,0,0,1,0,0,0,0
1,DwellSegmentationTimeFilter,1.619197e+09,2021-04-23 11:04:53-06:00,"{'type': 'Point', 'coordinates': [-105.0776493...",60830eac2b9762e085f774e8,1.619196e+09,2021-04-23 10:45:48.219950-06:00,"{'type': 'Point', 'coordinates': [-105.1228294...",19.079667,4764.133777,...,1,1,0,0,0,1,0,0,0,0
2,DwellSegmentationTimeFilter,1.619728e+09,2021-04-29 14:23:25-06:00,"{'type': 'Point', 'coordinates': [-105.0776309...",608b21db654c4a5ab2fa435a,1.619727e+09,2021-04-29 14:05:50.979267-06:00,"{'type': 'Point', 'coordinates': [-105.1228283...",17.567012,4731.604069,...,9,1,0,0,0,1,0,0,0,0
3,DwellSegmentationTimeFilter,1.619794e+09,2021-04-30 08:54:03-06:00,"{'type': 'Point', 'coordinates': [-105.0775022...",608c1f002f548f5db71d16ef,1.619793e+09,2021-04-30 08:34:49.162054-06:00,"{'type': 'Point', 'coordinates': [-105.1226652...",19.230632,4675.329725,...,9,1,0,0,0,1,0,0,0,0
4,DwellSegmentationTimeFilter,1.619807e+09,2021-04-30 12:15:42-06:00,"{'type': 'Point', 'coordinates': [-105.1228054...",608c8f7fb2d3b88f6947765e,1.619805e+09,2021-04-30 11:58:07.194775-06:00,"{'type': 'Point', 'coordinates': [-105.0960769...",17.580087,2855.830523,...,1,1,0,0,0,1,0,0,0,0


In [107]:
# def compute_alt_durations(df: pd.DataFrame, reverse_mapper: dict, duration_mapper: dict):
#     '''
#     We re-iterate over the generated available mode df and populate the modes.
#     We are only concerned with the rows that have primary_mode = 0, which indicates that they are alternate mode rows.
#     '''

#     column_names = list(reverse_mapper.values())

#     # Start with a 0 cost for everyone.
#     df[column_names] = 0

#     rows = list()
#     for ix, row in df.iterrows():
        
#         # Convert to a dict
#         row_dict = row.to_dict()

#         for mode in column_names:
#             if mode == reverse_mapper[row['chosen']]:
#                 row_dict[mode] = row_dict['section_distance_argmax']
#                 continue
            
#             mode_params = duration_mapper[mode]
#             # use availability as a mask.
#             row_dict[mode] = mode_params['intercept'] + (mode_params['coef'] * row_dict['section_distance_argmax'])
        
#         rows.append(row_dict)
    
#     return pd.DataFrame(rows).rename(columns=dict([(c, 'tt_'+c) for c in column_names]), inplace=False)

In [108]:
# av_time_df = compute_alt_durations(av_df, reverse_ordinal_mapping, duration_dict)

## Cost estimation

```
{'p_micro': 1, 'no_trip': 2, 's_car': 3, 'transit': 4, 'car': 5, 's_micro': 6, 'ridehail': 7, 'walk': 8, 'unknown': 9}
```

In [109]:
# All values are taken from VTPI.
# https://www.vtpi.org/tca/tca0501.pdf
mode_cost_per_mile = {
    # bicycle/skateboard
    'p_micro': 0.,
    'no_trip': 0.,
    # Shared car is half the cost of regular car, which is $0.6/mile.
    's_car': 0.3,
    # Rental car.
    'car': 0.6,
    # Average of bus and train taken.
    'transit': 0.5,
    # Shared bicyle or scooter - values taken from https://nacto.org/shared-micromobility-2020-2021/ and 
    # https://www.mckinsey.com/industries/automotive-and-assembly/our-insights/how-sharing-the-road-is-likely-to-transform-american-mobility
    's_micro': 0.3,
    # uber/taxi/lyft
    'ridehail': 2.,
    'walk': 0.,
    'unknown': 0.
}

# Assumptions.
mode_init_cost = {
    'p_micro': 0.,
    'no_trip': 0.,
    # Shared car is half the cost of regular car, which is $0.6/mile.
    's_car': 0.,
    # Rental car.
    'car': 0.,
    # Average of bus and train taken.
    'transit': 0.,
    # $1 unlocking cost.
    's_micro': 1.,
    # uber/taxi/lyft
    'ridehail': 1.,
    'walk': 0.,
    'unknown': 0.
}

In [110]:
def compute_cost_estimates(df: pd.DataFrame, cost_mapper: dict, init_cost_mapper: dict, ordinal_mapper: dict):
    
    # Create some extra colums.
    columns = list(ordinal_mapper.keys())

    # Initialize the columns to 0.
    df[columns] = 0.

    rows = list()

    # Iterate over every row.
    for _, row in df.iterrows():
        # Check which flags are active.
        row_dict = row.to_dict()

        # Access the section_distance_argmax attribute for the distance. Note that this is now in miles.
        distance = row_dict['section_distance_argmax']
        
        # Mask using availability.
        for lookup in columns:
            row_dict[lookup] = row_dict['av_' + lookup] * (init_cost_mapper[lookup] + (cost_mapper[lookup] * distance))

        rows.append(row_dict)

    new_df = pd.DataFrame(rows)
    new_df.rename(columns=dict([(c, 'cost_'+c) for c in ordinal_mapper.keys()]), inplace=True)

    return new_df

In [111]:
cost_df = compute_cost_estimates(
    av_df, cost_mapper=mode_cost_per_mile, 
    init_cost_mapper=mode_init_cost, 
    ordinal_mapper=ordinal_mapping
)

In [113]:
cost_df[[c for c in cost_df.columns if 'cost_' in c]].describe()

,cost_p_micro,cost_no_trip,cost_s_car,cost_transit,cost_car,cost_s_micro,cost_ridehail,cost_walk,cost_unknown
count,46053.0,46053.0,46053.000000,46053.000000,46053.000000,46053.000000,46053.000000,46053.0,46053.0
mean,0.0,0.0,0.759602,0.924603,1.354917,0.399658,5.604421,0.0,0.0
std,0.0,0.0,0.811586,1.228051,1.688835,0.770715,5.739880,0.0,0.0
min,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
25%,0.0,0.0,0.068502,0.000000,0.000000,0.000000,0.000000,0.0,0.0
50%,0.0,0.0,0.565642,0.508474,0.890892,0.000000,4.569688,0.0,0.0
75%,0.0,0.0,1.112221,1.361175,2.178041,0.000000,8.366328,0.0,0.0
max,0.0,0.0,12.867572,21.445953,25.735143,5.351574,86.783811,0.0,0.0


In [114]:
# student_status = {
#     'Not a student': 0,
#     'Yes - Full Time College/University': 1,
#     'Yes - Vocation/Technical/Trade School': 1,
#     'Yes - Part-Time College/University': 1,
#     'Fire Fighter 2 Training': 0,
#     'Taking prerequisites missing for grad program ': 1,
#     'Work': 0,
#     'Graduate': 1,
#     'Work at csu': 0,
#     'Custodian': 0, 
#     'taking classes toward early childhood licensure': 1,
#     'Prefer not to say': 0
# }


# cost_df['is_student'] = cost_df['is_student'].apply(lambda x: student_status[x])

In [115]:
# cost_df['age'] = cost_df['age'].apply(lambda x: x if x < 100 else 2024 - x)
# cost_df['n_working_residents'] = cost_df['n_working_residents'].apply(lambda x: 0 if x < 0 else x)
# cost_df.rename(
#     columns={'start_local_dt_weekday': 'start:DOW', 'end_local_dt_weekday': 'end:DOW'},
#     inplace=True
# )

In [116]:
cost_df.to_csv('../data/ReplacedMode_Fix_02142024.csv', index=False)